In [12]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [18]:
# 모델 정의
class PINN(tf.keras.Model):
    def __init__(self, hidden_size):
        super(PINN, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden_size)
        self.linear2 = tf.keras.layers.Dense(hidden_size)
        self.linear3 = tf.keras.layers.Dense(1)

    def call(self, x, t):
        inputs = tf.concat([x, t], axis=-1)
        outputs = tf.tanh(self.linear1(inputs))
        outputs = tf.tanh(self.linear2(outputs))
        outputs = self.linear3(outputs)
        return outputs

In [19]:
# 손실 함수
@tf.function
def loss_fn(model, x, t, alpha):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)

        u = model(x, t)
        u_t = tape.gradient(u, t)
        u_x = tape.gradient(u, x)
        u_xx = tape.gradient(u_x, x)

    del tape  # tape 삭제
    pde_loss = tf.reduce_mean(tf.square(u_t - alpha * u_xx))

    # 경계 조건 손실
    x_bc = tf.constant([[0.0], [1.0]], dtype=tf.float32)
    t_bc = tf.constant([[0.0], [0.0]], dtype=tf.float32)
    u_bc = model(x_bc, t_bc)
    bc_loss = tf.reduce_mean(tf.square(u_bc))

    # 초기 조건 손실
    x_ic = tf.reshape(tf.linspace(0.0, 1.0, 101), shape=(-1, 1))
    t_ic = tf.zeros_like(x_ic)
    u_ic = model(x_ic, t_ic)
    ic_loss = tf.reduce_mean(tf.square(u_ic - tf.sin(np.pi * x_ic)))

    loss = pde_loss + bc_loss + ic_loss
    return loss, (pde_loss, bc_loss, ic_loss)

In [20]:
# 하이퍼파라미터
alpha = 1.0
hidden_size = 20

# 모델 및 옵티마이저 초기화
model = PINN(hidden_size)
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# 학습 데이터 생성 (TensorFlow 텐서로 변환)
x = tf.reshape(tf.linspace(0.0, 1.0, 100), shape=(-1, 1))
t = tf.reshape(tf.linspace(0.0, 1.0, 100), shape=(-1, 1))

# 학습 루프
@tf.function  # 그래프 모드로 컴파일하여 성능 향상
def train_step(model, optimizer, x, t):
    with tf.GradientTape() as tape:
        loss, aux = loss_fn(model, x, t, alpha)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, aux

n_epochs = 10000
for epoch in range(1, n_epochs + 1):
    loss, aux = train_step(model, optimizer, x, t)
    pde_loss, bc_loss, ic_loss = aux

    if epoch % (n_epochs // 10) == 0:
        print(f"[{epoch:5d}/{n_epochs}] Loss: {loss.numpy():.3e} "
              f"pde: {pde_loss.numpy():.3e} bc: {bc_loss.numpy():.3e} ic: {ic_loss.numpy():.3e}")

[ 1000/10000] Loss: 2.601e-02 pde: 4.174e-03 bc: 5.788e-03 ic: 1.605e-02
[ 2000/10000] Loss: 3.264e-03 pde: 1.371e-04 bc: 5.053e-04 ic: 2.621e-03
[ 3000/10000] Loss: 2.301e-04 pde: 9.166e-05 bc: 2.131e-05 ic: 1.172e-04
[ 4000/10000] Loss: 4.665e-05 pde: 1.795e-05 bc: 2.422e-06 ic: 2.627e-05
[ 5000/10000] Loss: 2.691e-05 pde: 6.074e-06 bc: 1.359e-06 ic: 1.948e-05
[ 6000/10000] Loss: 5.699e-04 pde: 5.509e-04 bc: 9.597e-07 ic: 1.807e-05
[ 7000/10000] Loss: 1.371e-05 pde: 2.870e-06 bc: 5.479e-07 ic: 1.029e-05
[ 8000/10000] Loss: 9.823e-06 pde: 2.144e-06 bc: 3.248e-07 ic: 7.355e-06
[ 9000/10000] Loss: 8.832e-06 pde: 3.260e-06 bc: 2.782e-07 ic: 5.295e-06
[10000/10000] Loss: 5.607e-06 pde: 1.318e-06 bc: 1.335e-07 ic: 4.156e-06
